In [13]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from denn import *
import pandas as pd
import seaborn as sns
import io #Used as buffer

In [14]:
nn_p=5
nn_w = 5
nn_tw = 1

# Functions

In [15]:
path= Path(f'../../data/cluster_results')
col_by_freq = True
col_palette = 'Set3'
pat = re.compile('.*/(exp\d)/(\w*)/nonn/freq([0-9\.]+)div(\w+)/(\w+)_\w+.csv')
decode_keys = ['experiment','function','freq','div','method']
# nn_pat = re.compile('.*/(exp\d)/(\w*)/nn/freq([0-9\.]+)\w+div([A-Za-z]+)/(\w+)_(\w+)_\w+.csv')
# nn_decode_keys = ['experiment','function','freq','div','method','replace_mech']

nn_pat = re.compile('.*/(exp\d)/(\w*)/nn/freq([0-9\.]+)nn_w(\d+)nn_p(\d+)\w+nn_tw(\d+)\w+div([A-Za-z]+)/(\w+)_(\w+)_\w+.csv')
nn_decode_keys = ['experiment','function','freq','nnw','nnp','nntw','div','method','replace_mech']#,


def get_nn_files(m): return list(path.glob(f'**/nn/**/*{m}.csv'))


def read_nn_csv(f,m):
    df = pd.read_csv(f)
    for k,v in zip(nn_decode_keys,nn_pat.search(str(f)).groups()): df[k] = v
    df['freq'] = df['freq'].astype(float)
    df['method'] = df['method'].str.replace('NNnorm_Worst', 'NN')
    df['method'] = df['method'] + '_' + df['div'] #'_' + df['replace_mech'] +
    
    df.drop(['replace_mech','div'], axis=1, inplace=True)
    df.rename({'0':m.upper(), m:m.upper()}, axis=1, inplace=True)
    return df

def get_data(m, normalize=False):
    nn_files = get_nn_files(m)
    data = pd.concat([read_nn_csv(f,m) for f in nn_files])
    data = data[data['nnw']==str(nn_w)]
    data = data[data['nntw']==str(nn_tw)]
    data = data[data['nnp']==str(nn_p)]
    if normalize:
        data_norm = (data.groupby(['experiment','function','freq','method'])[m.upper()].mean().reset_index()
                         .groupby(['experiment','function'])[m.upper()].min().reset_index()
                         .rename({m.upper():m.upper()+'_norm'}, axis=1))
        data = data.merge(data_norm, 'left')
        data[m.upper()+'_norm'] = data[m.upper()] / data[m.upper()+'_norm']
    return data




# Table

In [16]:
df=get_data('nn_time', normalize=False)
df['function']=df['function'].str.title()
df.head()

,NN_TIME,experiment,function,freq,nnw,nnp,nntw,method
0,0.011216,exp1,Sphere,10.0,5,5,1,NN_RI
1,0.010989,exp1,Sphere,10.0,5,5,1,NN_RI
2,0.011096,exp1,Sphere,10.0,5,5,1,NN_RI
3,0.010941,exp1,Sphere,10.0,5,5,1,NN_RI
4,0.010812,exp1,Sphere,10.0,5,5,1,NN_RI


In [17]:
table2=df.pivot_table(values='NN_TIME', index=['function','method'] ,columns=['experiment','freq'])
table2

experiment             exp1      exp2      exp3      exp4
freq                   10.0      10.0      10.0      10.0
function   method                                        
Rastrigin  NN_HMu  0.010959  0.011167  0.011176  0.011068
           NN_No   0.011099  0.011473  0.011180  0.010543
           NN_RI   0.010993  0.011347  0.011072  0.010655
           NN_Rst  0.010721  0.010831  0.010719  0.010588
Rosenbrock NN_HMu  0.011019  0.011145  0.011156  0.011226
           NN_No   0.011167  0.011317  0.011020  0.010793
           NN_RI   0.010978  0.011183  0.010875  0.010873
           NN_Rst  0.010736  0.010638  0.010981  0.010642
Sphere     NN_HMu  0.010988  0.011076  0.011088  0.011077
           NN_No   0.011115  0.010982  0.011155  0.011326
           NN_RI   0.010966  0.011099  0.010874  0.010996
           NN_Rst  0.010937  0.010720  0.010792  0.010747

In [18]:
print(table2.to_csv(float_format='%.3f'))

experiment,,exp1,exp2,exp3,exp4
freq,,10.0,10.0,10.0,10.0
function,method,,,,
Rastrigin,NN_HMu,0.011,0.011,0.011,0.011
Rastrigin,NN_No,0.011,0.011,0.011,0.011
Rastrigin,NN_RI,0.011,0.011,0.011,0.011
Rastrigin,NN_Rst,0.011,0.011,0.011,0.011
Rosenbrock,NN_HMu,0.011,0.011,0.011,0.011
Rosenbrock,NN_No,0.011,0.011,0.011,0.011
Rosenbrock,NN_RI,0.011,0.011,0.011,0.011
Rosenbrock,NN_Rst,0.011,0.011,0.011,0.011
Sphere,NN_HMu,0.011,0.011,0.011,0.011
Sphere,NN_No,0.011,0.011,0.011,0.011
Sphere,NN_RI,0.011,0.011,0.011,0.011
Sphere,NN_Rst,0.011,0.011,0.011,0.011

